### evaluating our algorithm

In [45]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

dfTest=pd.read_csv('testTransactions.csv')
df=pd.read_csv('trainTransactions.csv')




# creating data frame to save the real values and the predicted values with the transaction number
allPredictions=pd.DataFrame(columns=['trans_num','is_fraud','prediction'])

for userid in dfTest['user_id'].unique():
    #print(f'user ID : {userid}')
    
    
    
    # split data
    userTrain=df[df['id']==userid]
    userTest=dfTest[dfTest['user_id']==userid]
    
    temp=userTest[userTest['user_id']==userid][['trans_num','is_fraud']]
    
    # Load from file
    with open(f'models123/{userid}.pkl', 'rb') as file:
        model = pickle.load(file)
        
    # predict the test set and save the result in a temporary datafram to merge with dataframe for all the results
    temp['prediction']=model.predict(userTest[['amt','unix_time','merch_lat','merch_long','minutes_from_midnight','hours_from_month_start']])
    frames = [allPredictions, temp]
    allPredictions = pd.concat(frames)
    

In [ ]:
'''
the results are
prediction : -1 means its fraud
prediction : 1 means its legitimate

and now we will convert them to 1 and 0
'''


### converting the results to 1 and 0 

In [61]:
allPredictions.loc[allPredictions.prediction == 1, 'prediction']=0

In [62]:
allPredictions.loc[allPredictions.prediction == -1, 'prediction']=1

In [63]:
allPredictions.head()


,trans_num,is_fraud,prediction
0,2da90c7d74bd46a0caf3777415b3ebd3,0,0
606,2ba65201d0c9119227e27b6640ff4415,0,0
4324,373feae82bc309c74b479ffd50b5934b,0,0
4340,844797fd827b566709aceea6ee8f29f2,0,1
4590,eca0b9de2d98820fb84119ad57a9c06c,0,0


### saving the results 

In [64]:
allPredictions.to_csv('allPredictions.csv')

### getting some percentages as a results

In [68]:

#accuracy For All transactions
len(allPredictions[((allPredictions['is_fraud']==1) & (allPredictions['prediction']==1)) | ((allPredictions['is_fraud']==0) & (allPredictions['prediction']==0))])/len(allPredictions)


0.6905298475761219

In [71]:
#TP accuracy of the real fraud transactions : that means from 100 fraud transaction , our algorithm will catch 85 
len(allPredictions[((allPredictions['is_fraud']==1) & (allPredictions['prediction']==1)) ])/len(allPredictions[allPredictions['is_fraud']==1])

0.850151362260343

In [72]:
#TN accuracy of the real legitimate transactions : that means from 100 legitimate transaction , our algorithm will predict correctly 65
len(allPredictions[((allPredictions['is_fraud']==0) & (allPredictions['prediction']==0)) ])/len(allPredictions[allPredictions['is_fraud']==0])

0.689958343419308